<a href="https://colab.research.google.com/github/JunjieLiang-Jack/Audio-Classifier/blob/main/5_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Classifying audio data with convolutional neural networks

<br/>
by Eduardo Garcia Rajo<br/>
<br/>

This notebook is part of the project "Urban sounds classification with Covnolutional Neural Networks" on [my Github](https://github.com/GorillaBus/urban-audio-classifier).<br/>
<br/>
Licensed under the GNU LESSER GENERAL PUBLIC LICENSE Version 3, 29 June 2007<br/>
<br/>

# UrbanSound8K Data Augmentation

On this notebook I'm creating variations of the original files applying the following effects:

1. Time stretch
2. Pitch shift
3. Noise

By default the new files are being saved in **UrbanSound8K/audio/augmented** with the same fold schema. At the end of the notebook a new metadata file is created combining the original dataset files and the generated ones.<br/>
<br/>
<br/>

In [ ]:
import os
import librosa
import librosa.display
import soundfile as sf
import pandas as pd
import numpy as np
import random
import matplotlib as plt
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### * Load metadata CSV

In [ ]:
# Set your path to the dataset
us8k_path = os.path.abspath('drive/MyDrive/CNN/Training Data/UrbanSound8K/')
audio_path = os.path.join(us8k_path, 'audio')
augmented_path = os.path.join(audio_path, 'augmented')

# Metadata
metadata_path = os.path.join(us8k_path, 'metadata/UrbanSound8K.csv')
metadata_augmented_path = os.path.abspath('drive/MyDrive/CNN/Training Data/UrbanSound8K/metadata/augmented-data.csv')

#### * Load metadata

In [ ]:
# Load the metadata from the generated CSV
metadata = pd.read_csv(metadata_path)

# Examine dataframe
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


#### 1. Time stretching (changing play time)

In [ ]:
rates = [0.81, 1.07]
total = len(metadata) * len(rates)
count = 0
for rate in rates: 
    # Generate new stretched audio file
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']
        
        # Speed sub-dir inside current fold dir
        curr_rate_path = augmented_path + '/fold' + curr_fold + '/speed_' + str(int(rate*100))

        
        # Create sub-dir if it does not exist
        if not os.path.exists(curr_rate_path):
            os.makedirs(curr_rate_path)
                    
        output_path = curr_rate_path + '/' + row['slice_file_name']
        
        # Skip when file already exists
        if (os.path.isfile(output_path)):
            count += 1 
            continue
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.time_stretch(y, rate=rate)
        sf.write(output_path, y_changed, sr)
        
        count += 1 
        
        clear_output(wait=True)
        print("Progress: {}/{}".format(count, total))
        print("Last file: ", row['slice_file_name'])


#### 2. Pitch shifting

In [ ]:
tone_steps = [-1, -2, 1, 2]
total = len(metadata) * len(tone_steps)
count = 0
for tone_step in tone_steps:
    # Generate new pitched audio
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

        # Pitch Shift sub-dir inside current fold dir
        curr_ps_path = augmented_path + '/fold' + curr_fold + '/pitch_' + str(tone_step)

        # Create sub-dir if it does not exist
        if not os.path.exists(curr_ps_path):
            os.makedirs(curr_ps_path)
        
        output_path = curr_ps_path + '/' + row['slice_file_name']
        
        # Skip when file already exists
        if (os.path.isfile(output_path)):
            count += 1 
            continue
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.pitch_shift(y, sr, n_steps=tone_step)
        sf.write(output_path, y_changed, sr)
        
        count += 1 
        
        clear_output(wait=True)
        print("Progress: {}/{}".format(count, total))
        print("Last file: ", row['slice_file_name'])

Progress: 34928/34928
Last file:  99812-1-6-0.wav


#### 3. Noise

In [ ]:
import random

def add_noise(data):
    noise = np.random.rand(len(data))
    noise_amp = random.uniform(0.005, 0.008)
    data_noise = data + (noise_amp * noise)
    return data_noise

total = len(metadata)
count = 0

# Generate new noised audio
for index, row in metadata.iterrows():        
    curr_fold = str(row['fold'])
    curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

    # Noised sub-dir inside current fold dir
    curr_noise_path = augmented_path + '/fold' + curr_fold + '/noise'

    # Create sub-dir if it does not exist
    if not os.path.exists(curr_noise_path):
        os.makedirs(curr_noise_path)
        
    output_path = curr_noise_path + '/' + row['slice_file_name']
        
    # Skip when file already exists
    if (os.path.isfile(output_path)):
        count += 1 
        continue
        
    y, sr = librosa.load(curr_file_path)  
    y_changed = add_noise(y)
    sf.write(output_path, y_changed, sr)
    
    count += 1 

    clear_output(wait=True)
    print("Progress: {}/{}".format(count, total))
    print("Last file: ", row['slice_file_name'])

Progress: 8732/8732
Last file:  99812-1-6-0.wav


#### 4. Create metadata for the new files

In [ ]:
def get_files_recursive(path):
    # create a list of file and sub directories names in the given directory 
    file_list = os.listdir(path)
    all_files = list()
    # Iterate over all the entries
    for entry in file_list:
        # Create full path
        full_path = os.path.join(path, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(full_path):
            all_files = all_files + get_files_recursive(full_path)
        else:
            all_files.append(full_path)
                
    return all_files   

In [ ]:
# Get every single file within the tree
files = get_files_recursive(augmented_path)

# Define metadata columns
names = []
classes = []
folds = []
augmentations = []

# Iterate and collect name, fold and class
for file in files:
    pieces = file.split("/")
    file = pieces[len(pieces) - 1]
    fold = pieces[len(pieces) - 3] 
    augment = pieces[len(pieces) - 2] 
    fold_num = fold[4:len(fold)]
    class_id = file.split("-")[1]

    # Push records
    names.append(file)
    folds.append(fold_num)
    classes.append(class_id)
    augmentations.append(augment)

# Create a dataframe with the new augmented data
new_meta = pd.DataFrame({'file': names, 'fold': folds, 'class_id': classes, 'augment': augmentations })

# Make sure class_id is int
new_meta['class_id'] = new_meta['class_id'].astype(np.int64)

print(len(new_meta), "new entries")

61124 new entries


In [ ]:
# Add class names to the new dataframe using merge
classes = pd.DataFrame({
    'class_id': range(0,10),
    'class': [
        'air_conditioner',
        'car_horn',
        'children_playing',
        'dog_bark',
        'drilling',
        'engine_idling',
        'gun_shot',
        'jackhammer',
        'siren',
        'street_music'
    ]
})

new_meta = pd.merge(new_meta, classes, on='class_id')


In [ ]:
new_meta.tail()

,file,fold,class_id,augment,class
61119,25039-6-0-0.wav,8,6,noise,gun_shot
61120,25039-6-1-0.wav,8,6,noise,gun_shot
61121,52284-6-0-0.wav,8,6,noise,gun_shot
61122,7065-6-0-0.wav,8,6,noise,gun_shot
61123,7068-6-0-0.wav,8,6,noise,gun_shot


#### 5. Integrate metadata in a single file

In [ ]:
# Modify original data to fit the new structure
del metadata['fsID'], metadata['start'], metadata['end'], metadata['salience']
metadata.columns = ['file', 'fold', 'class_id', 'class']
metadata['augment'] = 'none'


In [ ]:
# Concat the two dataframes
full_meta = pd.concat([metadata, new_meta])

# Verify lengths
if (len(full_meta) == len(metadata) + len(new_meta)):
    print("Dataframes merged correctly!")
else:
    print("Error! Lengths do not match.")

print("Initial data:", len(metadata))
print("New data:", len(new_meta))
print("Merged data:", len(full_meta))

Dataframes merged correctly!
Initial data: 8732
New data: 61124
Merged data: 69856


####  6. Save the new dataset

In [ ]:
# Save the new metadata
full_meta.to_csv(metadata_augmented_path, index=False, encoding="utf-8")